# Compute NPP Difference Percentiles

This notebook computes global percentiles (5% steps) for NPP difference scoring.

**Purpose:**
The percentiles are used in functional integrity scoring to scale absolute NPP difference with a truncated linear score (lower difference = higher score). We currently use p95 (and optionally p90), but store the full distribution for flexibility.

**When to run:**
- After retraining the NPP model
- After updating natural NPP predictions
- When observed NPP data source changes

**Output:**
Exports percentiles to a GEE asset at `NPP_DIFF_PERCENTILES_ASSET_PATH` (see settings.py).

In [1]:
import datetime

import ee

ee.Initialize()

from eii._utils.gee import load_tiled_collection
from eii.compute.settings import (
    NATURAL_NPP_ASSET_PATH,
    OBSERVED_NPP_ASSET_PATH,
    OBSERVED_NPP_YEAR_RANGE,
    SPATIAL_RESOLUTION,
)
from eii.utils.gee import create_assets_folder

In [2]:
NATURAL_NPP_ASSET_PATH

'projects/landler-open-data/assets/eii/intermediate/functional/predictions/natural_npp_v1'

In [4]:
potential_npp = ee.Image(NATURAL_NPP_ASSET_PATH).select("natural_npp_mean")
observed_collection = load_tiled_collection(OBSERVED_NPP_ASSET_PATH).filterDate(
    OBSERVED_NPP_YEAR_RANGE[0], OBSERVED_NPP_YEAR_RANGE[1]
)
actual_npp = observed_collection.select([0]).mean()
npp_difference = actual_npp.subtract(potential_npp).abs().rename("npp_difference")

In [16]:
NUM_SAMPLES = 10000
SCALE = SPATIAL_RESOLUTION
SEED = 42
PERCENTILE_ASSET_PATH = (
    "projects/landler-open-data/assets/eii/intermediate/functional/npp_diff_decile_breaks"
)

# Global land bounds (exclude oceans and polar regions)
land_bounds = ee.Geometry.Rectangle([-180, -60, 180, 75], proj="EPSG:4326", geodesic=False)

# Land mask from MODIS (class 17 = water)
modis_lc = ee.ImageCollection("MODIS/061/MCD12Q1").first()
land_mask = modis_lc.select("LC_Type1").neq(17)
npp_difference_land = npp_difference.updateMask(land_mask)

samples = npp_difference_land.sample(
    region=land_bounds,
    scale=SCALE,
    numPixels=NUM_SAMPLES,
    seed=SEED,
    dropNulls=True,
    tileScale=8,
)

In [17]:
# Compute count + percentiles server-side
percentiles = list(range(5, 100, 5))
combined_reducer = ee.Reducer.count().combine(ee.Reducer.percentile(percentiles), sharedInputs=True)
stats = samples.reduceColumns(reducer=combined_reducer, selectors=["npp_difference"])

# Build Feature with percentile breaks as properties (dummy point geometry required for export)
properties = ee.Dictionary({f"p{p}": stats.get(f"p{p}") for p in percentiles}).combine(
    ee.Dictionary(
        {
            "sample_count": stats.get("count"),
            "computed_date": datetime.datetime.now().strftime("%Y-%m-%d"),
            "observed_npp_start": OBSERVED_NPP_YEAR_RANGE[0],
            "observed_npp_end": OBSERVED_NPP_YEAR_RANGE[1],
            "natural_npp_source": NATURAL_NPP_ASSET_PATH,
            "observed_npp_source": OBSERVED_NPP_ASSET_PATH,
            "scale_m": SCALE,
            "num_samples_requested": NUM_SAMPLES,
        }
    )
)
dummy_point = ee.Geometry.Point([0, 0])
percentile_fc = ee.FeatureCollection([ee.Feature(dummy_point, properties)])

In [ ]:
create_assets_folder("/".join(PERCENTILE_ASSET_PATH.split("/")[:-1]))
try:
    ee.data.getAsset(PERCENTILE_ASSET_PATH)
    raise RuntimeError(f"Asset already exists: {PERCENTILE_ASSET_PATH}. Delete it first.")
except ee.EEException:
    task = ee.batch.Export.table.toAsset(
        collection=percentile_fc,
        description="NPP_Diff_Percentiles",
        assetId=PERCENTILE_ASSET_PATH,
    )
    task.start()

In [20]:
task.status()

{'state': 'COMPLETED',
 'description': 'NPP_Diff_Percentiles',
 'priority': 100,
 'creation_timestamp_ms': 1768514107726,
 'update_timestamp_ms': 1768514204601,
 'start_timestamp_ms': 1768514116959,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/landler-open-data/assets/eii/intermediate/functional/npp_diff_decile_breaks'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 2197.875244140625,
 'id': 'ZBBZVHOEZX3BSPLPSFX4BCZE',
 'name': 'projects/science-428407/operations/ZBBZVHOEZX3BSPLPSFX4BCZE'}

## 4. Verify Results (after export completes)

In [21]:
result_fc = ee.FeatureCollection(PERCENTILE_ASSET_PATH)
result_fc.first().getInfo()["properties"]

{'computed_date': '2026-01-15',
 'natural_npp_source': 'projects/landler-open-data/assets/eii/intermediate/functional/predictions/natural_npp_v1',
 'num_samples_requested': 10000,
 'observed_npp_end': '2025-01-01',
 'observed_npp_source': 'projects/landler-open-data/assets/datasets/clms/npp/annual',
 'observed_npp_start': '2022-01-01',
 'p10': 28479.605668777684,
 'p15': 95948.12880735763,
 'p20': 164307.88207538315,
 'p25': 294455.11794354836,
 'p30': 361843.1801635304,
 'p35': 492020.5090751265,
 'p40': 622068.7266666667,
 'p45': 750126.7559185607,
 'p5': 28479.605668777684,
 'p50': 882509.3822674417,
 'p55': 1076025.6100543477,
 'p60': 1283575.8124999998,
 'p65': 1478865.338425926,
 'p70': 1668378.8773148146,
 'p75': 1937024.2743055557,
 'p80': 2328698.6375,
 'p85': 2788236.575396826,
 'p90': 3501608.428571429,
 'p95': 4814612.838541667,
 'sample_count': 2813,
 'scale_m': 300}